In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('/home/rwu/cargospace/result.txt',sep = '\t')
df

,Protein_name,Sample,cancer
0,AHR_AHR_HUMAN.H11MO.0.B,0QJ6J2T_sample001,0QJ6J2T
1,AIRE_AIRE_HUMAN.H11MO.0.C,0QJ6J2T_sample001,0QJ6J2T
2,ANDR_ANDR_HUMAN.H11MO.0.A,0QJ6J2T_sample001,0QJ6J2T
3,AP2A_AP2A_HUMAN.H11MO.0.A,0QJ6J2T_sample001,0QJ6J2T
4,AP2B_AP2B_HUMAN.H11MO.0.B,0QJ6J2T_sample001,0QJ6J2T
...,...,...,...
24042,ZNF547_M07600_2.00,ZF8WDFS_sample007,ZF8WDFS
24043,ZNF652_MA1657.1,ZF8WDFS_sample007,ZF8WDFS
24044,ZNF8_M08401_2.00,ZF8WDFS_sample007,ZF8WDFS
24045,ZSCAN29_M08271_2.00,ZF8WDFS_sample007,ZF8WDFS


In [7]:
len(set(df['Protein_name']))

1356

In [ ]:
count = 0
for protein in list(set(df['Protein_name'])):
    count_cancer = 0
    print(f"------{count}/1350------")
    #protein = df['Protein_name'][i]
    new_protein = True
    #print("----------------" + protein + "----------------")
    df_proetin = df[df['Protein_name'] == protein]
    cancers_samples = df_proetin['Sample'].tolist()
    print(len(cancers_samples))
    
    for cancer in cancers_samples:
        print("Processing: " + cancer)
        path = "/home/zlei/cargospace/merged_data1/" + cancer.split("_")[0] + ".txt"
        df_cancer = pd.read_csv(path,sep='\t')
        df_cancer_temp = df_cancer[df_cancer['Protein name'] == protein]
        
        all_cancer_columns = [col for col in df_cancer_temp.columns if cancer.split("_")[0] in col]
        correlated_cancer_columns = [col for col in df_cancer_temp.columns if cancer in col]
        if all_cancer_columns != correlated_cancer_columns:
            additional_list = [element for element in all_cancer_columns if element not in correlated_cancer_columns]
            print("\n!!!!!Have additional columns!!!!!")
            #print(additional_list)
            df_cancer_temp = df_cancer_temp.drop(additional_list,axis = 1)
        
        if new_protein == True:
            df_cancer_protein = df_cancer_temp
            df_cancer_protein['Cancer name'] = cancer
            rename_columns_fp = cancer + '_fp_score'
            rename_columns_vaf = cancer + '_VAF'
            df_cancer_protein.rename(columns={rename_columns_fp: 'fp_score',rename_columns_vaf:'VAF'}, inplace=True)
            new_protein = False
        else:
            df_cancer_temp['Cancer name'] = cancer
            rename_columns_fp = cancer + '_fp_score'
            rename_columns_vaf = cancer + '_VAF'
            df_cancer_temp.rename(columns={rename_columns_fp: 'fp_score',rename_columns_vaf:'VAF'}, inplace=True)
            df_cancer_protein = pd.concat([df_cancer_protein, df_cancer_temp],axis=0)
            
        df_cancer_protein.reset_index(drop=True, inplace=True)
        
        
        save_path = '/home/zlei/cargospace/protein_caner_5_6/' + protein + '.csv'
        
        if os.path.exists(save_path):
            os.remove(save_path)
        df_cancer_protein.to_csv(save_path)
    count += 1
    

In [8]:
def list_files(directory):
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return files

files = list_files('/home/zlei/cargospace/protein_caner_5_6/')
#files = list_files('/home/zlei/cargospace/protein_of_each_cancer/')

In [9]:
df_save = pd.read_csv('/home/zlei/cargospace/protein_caner_5_6/'+files[0], index_col = 0)
print(files[0],len(df_save))

for file in files[1:]:
    temp_df = pd.read_csv('/home/zlei/cargospace/protein_caner_5_6/'+ file, index_col = 0)
    df_save = pd.concat([df_save, temp_df],axis=0)
    df_save.reset_index(drop=True, inplace=True)


HXB7_HXB7_HUMAN.H11MO.0.C.csv 90


In [10]:
df_save

,TFBS_chr,TFBS_start,TFBS_end,TFBS_strand,TFBS_score,fp_score,[2]POS,[3]REF,[4]ALT,VAF,Protein name,Cancer name
0,chr13,37201771,37201781,+,2.424468,0.021706,37201772,T,C,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
1,chr1,93106435,93106445,+,1.849734,0.025261,93106438,G,A,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
2,chr10,27367239,27367249,+,2.424468,0.051675,27367245,T,C,0.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
3,chr1,95575724,95575734,+,2.274308,0.015278,95575733,T,C,0.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
4,chr19,54283056,54283066,-,1.849734,0.105843,54283057,A,G,0.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
...,...,...,...,...,...,...,...,...,...,...,...,...
1757794,chr6,35797961,35797973,-,3.196945,0.052567,35797962,T,C,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1757795,chr1,154851751,154851763,-,3.214737,0.053391,154851762,G,A,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1757796,chr11,117734381,117734393,+,4.268656,0.086781,117734383,C,T,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1757797,chr9,18680196,18680208,-,2.732011,0.021289,18680197,T,C,1.0,ZNF254_M07760_2.00,O697I0L_sample001


In [13]:
df_save_VAF1.to_csv('5_7_merged_data_VAF1.csv')

In [12]:
df_save_VAF1 = df_save[df_save['VAF'] == 1.0]
df_save_VAF1.reset_index(drop=True, inplace=True)
df_save_VAF1

,TFBS_chr,TFBS_start,TFBS_end,TFBS_strand,TFBS_score,fp_score,[2]POS,[3]REF,[4]ALT,VAF,Protein name,Cancer name
0,chr13,37201771,37201781,+,2.424468,0.021706,37201772,T,C,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
1,chr1,93106435,93106445,+,1.849734,0.025261,93106438,G,A,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,1KEQR84_sample003
2,chr9,87524647,87524657,-,2.274308,0.036110,87524654,T,G,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,3CA9B3H_sample001
3,chr18,2594543,2594553,-,1.849734,0.048688,2594550,T,C,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,3CA9B3H_sample001
4,chr15,44046059,44046069,+,1.849734,0.033008,44046065,T,A,1.0,HXB7_HXB7_HUMAN.H11MO.0.C,3CA9B3H_sample001
...,...,...,...,...,...,...,...,...,...,...,...,...
1080718,chr6,35797961,35797973,-,3.196945,0.052567,35797962,T,C,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1080719,chr1,154851751,154851763,-,3.214737,0.053391,154851762,G,A,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1080720,chr11,117734381,117734393,+,4.268656,0.086781,117734383,C,T,1.0,ZNF254_M07760_2.00,O697I0L_sample001
1080721,chr9,18680196,18680208,-,2.732011,0.021289,18680197,T,C,1.0,ZNF254_M07760_2.00,O697I0L_sample001


In [16]:
df_cancer_protein = pd.read_csv('/home/zlei/cargospace/5_7_merged_data_VAF1.csv',index_col=0)
df_cancer_protein_VAF1 = df_cancer_protein[df_cancer_protein['VAF'] == 1.0]
for cancer in set(df_cancer_protein_VAF1['Cancer name']):
    #print(cancer)
    temp_df_VAF1 = df_cancer_protein_VAF1[df_cancer_protein_VAF1['Cancer name'] == cancer]
    save_path = '/home/zlei/cargospace/cancer_5_7_VAF1/' + cancer + '.csv'
            
    if os.path.exists(save_path):
        os.remove(save_path)
    temp_df_VAF1.to_csv(save_path)

In [15]:
df_cancer_protein = pd.read_csv('/home/zlei/cargospace/5_7_merged_data_VAF1.csv',index_col=0)
df_cancer_protein_VAF1 = df_cancer_protein[df_cancer_protein['VAF'] == 1.0]
for protein in set(df_cancer_protein_VAF1['Protein name']):
    temp_df_VAF1 = df_cancer_protein_VAF1[df_cancer_protein_VAF1['Protein name'] == protein]
    save_path = '/home/zlei/cargospace/protein_5_7_VAF1/' + protein + '.csv'
            
    if os.path.exists(save_path):
        os.remove(save_path)
    temp_df_VAF1.to_csv(save_path)